## Setup and Installation

In [44]:
# Install required packages
!pip install -q -U google-generativeai pandas


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [45]:
import os
import pandas as pd
import google.generativeai as genai
from enum import Enum
from typing import Dict, List
import json
import time
from pathlib import Path

In [46]:
# Language Configuration
class Language(Enum):
    RUSSIAN = {"code": "RU", "name": "Russian", "native": "Русский"}
    FRENCH = {"code": "FR", "name": "French", "native": "Français"}
    CHINESE = {"code": "ZH", "name": "Chinese", "native": "中文"}
    ARABIC = {"code": "AR", "name": "Arabic", "native": "العربية"}

# ===========================
# SELECT TARGET LANGUAGE HERE
# ===========================
# TARGET_LANGUAGE = Language.RUSSIAN
# TARGET_LANGUAGE = Language.FRENCH
TARGET_LANGUAGE = Language.CHINESE
# TARGET_LANGUAGE = Language.ARABIC

print(f"Target Language: {TARGET_LANGUAGE.value['name']} ({TARGET_LANGUAGE.value['native']})")
print(f"Language Code: {TARGET_LANGUAGE.value['code']}")

Target Language: Chinese (中文)
Language Code: ZH


In [47]:
# Configure Gemini API
# Set your API key as environment variable: GOOGLE_API_KEY
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    print("⚠️ WARNING: GOOGLE_API_KEY not found in environment variables!")
    print("Please set it using: os.environ['GOOGLE_API_KEY'] = 'your-api-key-here'")
else:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✓ Gemini API configured successfully")

✓ Gemini API configured successfully


In [48]:
# Initialize Gemini Model
MODEL_NAME = 'gemini-2.0-flash'
model = genai.GenerativeModel(MODEL_NAME)

print(f"Using model: {MODEL_NAME}")

Using model: gemini-2.0-flash


In [49]:
# Load English game data
input_csv = "EN/disinformer_full_games_clues.csv"

if not os.path.exists(input_csv):
    print(f"❌ Error: Input file not found: {input_csv}")
else:
    df = pd.read_csv(input_csv)
    print(f"✓ Loaded {len(df)} rows from {input_csv}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nFirst few rows:")
    display(df.head())

✓ Loaded 3000 rows from EN/disinformer_full_games_clues.csv

Columns: ['test_run', 'topic_category', 'round', 'answer', 'choices', 'clue_type', 'clue_number', 'clue_text', 'word_count', 'length_ok', 'manual_score / comment']

First few rows:


,test_run,topic_category,round,answer,choices,clue_type,clue_number,clue_text,word_count,length_ok,manual_score / comment
0,1,Books,1,Fantasy,"Fantasy, Sci-Fi, Adventure",informed,1,"This genre often features magic, mythical crea...",16,YES,NaN
1,1,Books,1,Fantasy,"Fantasy, Sci-Fi, Adventure",informed,2,"It typically involves quests, battles against ...",18,YES,NaN
2,1,Books,1,Fantasy,"Fantasy, Sci-Fi, Adventure",informed,3,The narrative often includes characters with s...,15,YES,NaN
3,1,Books,1,Fantasy,"Fantasy, Sci-Fi, Adventure",informed,4,"These narratives often feature heroes, their j...",20,YES,NaN
4,1,Books,1,Fantasy,"Fantasy, Sci-Fi, Adventure",informed,5,Readers are often transported to realms where ...,17,YES,NaN


In [50]:
# Define translation prompt template
def create_translation_prompt(target_lang_info: Dict[str, str]) -> str:
    """
    Create a system prompt for translation.
    
    Args:
        target_lang_info: Dictionary with language 'code', 'name', and 'native' name
    
    Returns:
        Formatted prompt string
    """
    return f"""You are a professional translator specializing in game content localization.

Your task is to translate Disinformer game clues from English to {target_lang_info['name']} ({target_lang_info['native']}).

IMPORTANT TRANSLATION STRATEGY:
- Answer and choices are translated ONCE per round and reused across all 15 clues in that round
- This ensures consistency: the same answer and choice options appear in all clues of a round
- You will either translate (1) answer+choices for a round, OR (2) only the clue_text (when answer/choices are already translated)

CORE REQUIREMENTS:
1. When translating answer + choices:
   - Translate the answer accurately to the target language
   - Translate each choice to the target language, preserving EXACT ORDER
   - Keep the same number of comma-separated choices
   
2. When translating only clue_text:
   - Preserve tone and intent based on clue type:
     * INFORMED clues: Accurate, helpful hints pointing to the correct answer
     * MISINFORMED clues: Vague, generic statements that create ambiguity, pointing to multiple options
     * FAKE clues: Point to the wrong answer options
     * EXTRA clues: Additional helpful clues for the correct answer
   
3. Keep proper nouns (names, titles, places) in their original form or use standard translations when appropriate

4. Ensure natural, fluent {target_lang_info['name']} that sounds native

5. Preserve game mechanics and clarity

VALIDATION CHECKLIST:
- All translated fields are NOT in English ✓
- All fields are valid JSON strings ✓
- When translating answer+choices: order is preserved and all fields in target language ✓

Return ONLY valid JSON with the requested fields. Do not include explanations or markdown formatting."""

TRANSLATION_PROMPT = create_translation_prompt(TARGET_LANGUAGE.value)
print("Translation prompt created:")
print("="*80)
print(TRANSLATION_PROMPT)
print("="*80)


Translation prompt created:
You are a professional translator specializing in game content localization.

Your task is to translate Disinformer game clues from English to Chinese (中文).

IMPORTANT TRANSLATION STRATEGY:
- Answer and choices are translated ONCE per round and reused across all 15 clues in that round
- This ensures consistency: the same answer and choice options appear in all clues of a round
- You will either translate (1) answer+choices for a round, OR (2) only the clue_text (when answer/choices are already translated)

CORE REQUIREMENTS:
1. When translating answer + choices:
   - Translate the answer accurately to the target language
   - Translate each choice to the target language, preserving EXACT ORDER
   - Keep the same number of comma-separated choices

2. When translating only clue_text:
   - Preserve tone and intent based on clue type:
     * INFORMED clues: Accurate, helpful hints pointing to the correct answer
     * MISINFORMED clues: Vague, generic statements

In [51]:
def translate_round_data(answer: str, choices: str, clue_text: str, clue_type: str) -> Dict[str, str]:
    """
    Translate a single clue text using Gemini. Answer and choices are translated separately.
    
    Args:
        answer: The correct answer (already translated)
        choices: Comma-separated choices (already translated)
        clue_text: The clue text to translate
        clue_type: Type of clue (informed/misinformed/fake/extra)
    
    Returns:
        Dictionary with translated clue_text
    """
    user_message = f"""Translate only the clue text to maintain consistency within a game round.

Answer (already translated): {answer}
Choices (already translated): {choices}
Clue Type: {clue_type}
Clue Text (MUST translate, 15-20 words): {clue_text}

IMPORTANT: 
- Translate ONLY the clue_text field
- The clue_text MUST be in the target language (15-20 words), NOT English
- Return only JSON with the clue_text field."""
    
    try:
        response = model.generate_content(
            [TRANSLATION_PROMPT, user_message],
            generation_config=genai.types.GenerationConfig(
                temperature=0.3,
                max_output_tokens=500,
            )
        )
        
        # Extract JSON from response
        response_text = response.text.strip()
        
        # Remove markdown code blocks if present
        if response_text.startswith('```'):
            response_text = response_text.split('```')[1]
            if response_text.startswith('json'):
                response_text = response_text[4:]
            response_text = response_text.strip()
        
        # Parse JSON
        translation = json.loads(response_text)
        
        # Validate required field
        if 'clue_text' not in translation:
            raise ValueError("Missing clue_text field in translation")
        
        return translation
        
    except Exception as e:
        print(f"❌ Translation error: {e}")
        print(f"Response: {response_text if 'response_text' in locals() else 'N/A'}")
        return {"clue_text": clue_text}

def translate_round_answers_and_choices(answer: str, choices: str) -> Dict[str, str]:
    """
    Translate answer and choices once per round.
    
    Args:
        answer: The correct answer
        choices: Comma-separated choices
    
    Returns:
        Dictionary with translated answer and choices (maintaining choice order)
    """
    user_message = f"""Translate answer and choices for a game round. These will be reused for all clues in this round.

Answer: {answer}
Choices: {choices}

CRITICAL REQUIREMENTS:
1. Translate the answer to the target language
2. Translate each choice to the target language, preserving the EXACT ORDER
3. Keep the same number of choices separated by commas
4. Return a JSON object with answer and choices fields

IMPORTANT:
- ALL fields MUST be in the target language, NOT English
- The order of choices MUST be preserved
- Return only valid JSON

Return ONLY a valid JSON object with this exact structure:
{{
  "answer": "translated answer (NOT in English)",
  "choices": "translated choice1, translated choice2, translated choice3 (NOT in English, SAME ORDER)"
}}"""
    
    try:
        response = model.generate_content(
            [TRANSLATION_PROMPT, user_message],
            generation_config=genai.types.GenerationConfig(
                temperature=0.3,
                max_output_tokens=300,
            )
        )
        
        # Extract JSON from response
        response_text = response.text.strip()
        
        # Remove markdown code blocks if present
        if response_text.startswith('```'):
            response_text = response_text.split('```')[1]
            if response_text.startswith('json'):
                response_text = response_text[4:]
            response_text = response_text.strip()
        
        # Parse JSON
        translation = json.loads(response_text)
        
        # Validate required fields
        if not all(k in translation for k in ['answer', 'choices']):
            raise ValueError("Missing required fields in translation")
        
        return translation
        
    except Exception as e:
        print(f"❌ Translation error: {e}")
        print(f"Response: {response_text if 'response_text' in locals() else 'N/A'}")
        return {
            "answer": answer,
            "choices": choices
        }

print("✓ Translation functions defined")


✓ Translation functions defined


In [52]:
def translate_dataframe(df: pd.DataFrame, delay_seconds: float = 2.0) -> pd.DataFrame:
    """
    Translate all rows in the dataframe, translating answer/choices once per round and reusing them.
    
    Args:
        df: Input dataframe with English content
        delay_seconds: Delay between API calls to avoid rate limits
    
    Returns:
        New dataframe with translated content
    """
    translated_rows = []
    total_rows = len(df)
    
    # Group by round to translate answers/choices once per round
    # Unique identifier is (topic_category, test_run, round)
    round_groups = df.groupby(['topic_category', 'test_run', 'round'], sort=False)
    total_groups = len(round_groups)
    
    print(f"Starting translation of {total_rows} rows across {total_groups} rounds...\n")
    
    group_idx = 0
    for (topic, test_run, round_num), group_df in round_groups:
        if group_idx % 10 == 0:
            print(f"Progress: {group_idx}/{total_groups} rounds ({group_idx/total_groups*100:.1f}%)")
        
        # Get the first row's answer and choices (same for all clues in this round)
        first_row = group_df.iloc[0]
        answer = first_row['answer']
        choices = first_row['choices']
        
        # Translate answer and choices once for the entire round
        round_translation = translate_round_answers_and_choices(answer, choices)
        translated_answer = round_translation['answer']
        translated_choices = round_translation['choices']
        
        # Rate limiting after round-level translation
        time.sleep(delay_seconds * 0.5)
        
        # Now translate each clue in this round
        for idx, row in group_df.iterrows():
            # Translate the clue text
            clue_translation = translate_round_data(
                answer=translated_answer,
                choices=translated_choices,
                clue_text=row['clue_text'],
                clue_type=row['clue_type']
            )
            
            # Create new row with translated content
            new_row = row.copy()
            new_row['answer'] = translated_answer
            new_row['choices'] = translated_choices
            new_row['clue_text'] = clue_translation['clue_text']
            
            # Update word count
            new_row['word_count'] = len(clue_translation['clue_text'].split())
            new_row['length_ok'] = 'YES' if 15 <= new_row['word_count'] <= 20 else 'NO'
            
            translated_rows.append(new_row)
            
            # Rate limiting between clues
            time.sleep(delay_seconds * 0.5)
        
        group_idx += 1
    
    print(f"\n✓ Translation complete: {total_groups}/{total_groups} rounds (100%)")
    return pd.DataFrame(translated_rows)

print("✓ Batch translation function defined")


✓ Batch translation function defined


In [53]:
# Test translation with a single round (all clues with same answer/choices)
print("Testing translation with first round (all clues should share same translated answer/choices)...\n")

# Get first round's data
first_round = df.iloc[0:3]  # Get first 3 clues (typically from same round)
test_row = first_round.iloc[0]

print(f"Original (same for all clues in this round):")
print(f"  Answer: {test_row['answer']}")
print(f"  Choices: {test_row['choices']}\n")

# Step 1: Translate answer and choices once
print("Step 1: Translating answer and choices...")
round_translation = translate_round_answers_and_choices(
    answer=test_row['answer'],
    choices=test_row['choices']
)
translated_answer = round_translation['answer']
translated_choices = round_translation['choices']
print(f"Translated (once, reused for all clues):")
print(f"  Answer: {translated_answer}")
print(f"  Choices: {translated_choices}\n")

# Step 2: Translate clue texts
print("Step 2: Translating clues (reusing translated answer/choices)...")
for i, row in first_round.iterrows():
    print(f"\n  Clue {i+1} ({row['clue_type']}):")
    print(f"    Original: {row['clue_text']}")
    
    clue_translation = translate_round_data(
        answer=translated_answer,
        choices=translated_choices,
        clue_text=row['clue_text'],
        clue_type=row['clue_type']
    )
    
    print(f"    Translated: {clue_translation['clue_text']}")
    print(f"    Word count: {len(clue_translation['clue_text'].split())}")


Testing translation with first round (all clues should share same translated answer/choices)...

Original (same for all clues in this round):
  Answer: Fantasy
  Choices: Fantasy, Sci-Fi, Adventure

Step 1: Translating answer and choices...


Translated (once, reused for all clues):
  Answer: 奇幻
  Choices: 奇幻, 科幻, 冒险

Step 2: Translating clues (reusing translated answer/choices)...

  Clue 1 (informed):
    Original: This genre often features magic, mythical creatures, and imaginative worlds that defy the laws of reality.
    Translated: 这个类型通常包含魔法、神话生物和富有想象力的世界，这些世界挑战现实世界的法则。
    Word count: 1

  Clue 2 (informed):
    Original: It typically involves quests, battles against evil, and the triumph of good over darkness in a fictional setting.
    Translated: 它通常包含任务、与邪恶的战斗，以及在虚构背景下善战胜恶的胜利。
    Word count: 1

  Clue 3 (informed):
    Original: The narrative often includes characters with special abilities, embarking on perilous journeys and overcoming challenges.
    Translated: 故事通常包含拥有特殊能力的角色，他们踏上危险的旅程并克服挑战，这是一种常见的叙事手法。
    Word count: 1


In [54]:
from enum import Enum

class GameTopic(Enum):
    BOOKS = "Books"
    BROADCAST_MEDIA = "Broadcast Media"
    FOOD = "Food"
    INVENTIONS = "Inventions"
    NATURE = "Nature"
    PLACES = "Places"
    SONGS = "Songs"
    SPORTS = "Sports"
    TECHNOLOGY = "Technology"
    VIDEO_GAMES = "Video Games"

    def __str__(self):
        return self.value

In [ ]:
# Translate all data
# WARNING: This will make many API calls and may take considerable time
# Consider translating in batches or filtering by topic first

# Option 1: Translate everything (uncomment to use)
# translated_df = translate_dataframe(df, delay_seconds=2.0)

# Option 2: Translate a specific topic (recommended for testing)
topic_to_translate = GameTopic.BOOKS.value
# topic_to_translate = GameTopic.BROADCAST_MEDIA.value
# topic_to_translate = GameTopic.FOOD.value
# topic_to_translate = GameTopic.INVENTIONS.value
# topic_to_translate = GameTopic.NATURE.value
# topic_to_translate = GameTopic.PLACES.value
# topic_to_translate = GameTopic.SONGS.value
# topic_to_translate = GameTopic.SPORTS.value
# topic_to_translate = GameTopic.TECHNOLOGY.value
# topic_to_translate = GameTopic.VIDEO_GAMES.value

df_topic = df[df['topic_category'] == topic_to_translate].copy()
print(f"Translating topic: {topic_to_translate} ({len(df_topic)} rows)\n")

translated_df = translate_dataframe(df_topic, delay_seconds=0.0)

Translating topic: Video Games (300 rows)

Starting translation of 300 rows across 20 rounds...

Progress: 0/20 rounds (0.0%)
Progress: 10/20 rounds (50.0%)

✓ Translation complete: 20/20 rounds (100%)


In [92]:
# Review translation statistics
print("Translation Statistics:")
print("="*80)
print(f"Total rows translated: {len(translated_df)}")
print(f"\nWord count distribution:")
print(translated_df['word_count'].value_counts().sort_index())
print(f"\nLength compliance:")
print(translated_df['length_ok'].value_counts())
print(f"\nClues by type:")
print(translated_df['clue_type'].value_counts())

# Show sample translations
print("\n" + "="*80)
print("Sample translations:")
print("="*80)
display(translated_df.head(10))

Translation Statistics:
Total rows translated: 300

Word count distribution:
word_count
1    300
Name: count, dtype: int64

Length compliance:
length_ok
NO    300
Name: count, dtype: int64

Clues by type:
clue_type
informed       180
fake            60
misinformed     40
extra           20
Name: count, dtype: int64

Sample translations:


,test_run,topic_category,round,answer,choices,clue_type,clue_number,clue_text,word_count,length_ok,manual_score / comment
2700,1,Video Games,1,角色扮演游戏,"角色扮演游戏, 冒险游戏, 策略游戏",informed,1,玩家通过获取经验、升级和获得新技能与物品来发展角色，是其核心机制。,1,NO,NaN
2701,1,Video Games,1,角色扮演游戏,"角色扮演游戏, 冒险游戏, 策略游戏",informed,2,玩家通常控制一个或一组角色，并做出影响故事走向的决策，体验扮演不同人生的乐趣。,1,NO,NaN
2702,1,Video Games,1,角色扮演游戏,"角色扮演游戏, 冒险游戏, 策略游戏",informed,3,注重故事和角色发展，让你探索丰富的世界，与非玩家角色互动，沉浸于引人入胜的剧情之中。,1,NO,NaN
2703,1,Video Games,1,角色扮演游戏,"角色扮演游戏, 冒险游戏, 策略游戏",informed,4,它通常包含玩家必须完成的任务、工作或使命，以推进游戏的故事叙述。,1,NO,NaN
2704,1,Video Games,1,角色扮演游戏,"角色扮演游戏, 冒险游戏, 策略游戏",informed,5,提供自由探索感，允许玩家发现新区域并揭开秘密，沉浸在丰富的叙事和角色发展中。,1,NO,NaN
2705,1,Video Games,1,角色扮演游戏,"角色扮演游戏, 冒险游戏, 策略游戏",informed,6,沉浸式世界通常通过融入奇幻、科幻或历史时期的元素来创造，让玩家扮演其中角色。,1,NO,NaN
2706,1,Video Games,1,角色扮演游戏,"角色扮演游戏, 冒险游戏, 策略游戏",informed,7,这种游戏方式允许玩家自定义角色，通过各种技能和属性来塑造独一无二的游戏体验。,1,NO,NaN
2707,1,Video Games,1,角色扮演游戏,"角色扮演游戏, 冒险游戏, 策略游戏",informed,8,它通常包含一个广阔的世界，玩家可以探索各种地点并发现隐藏的物品，提升游戏体验。,1,NO,NaN
2708,1,Video Games,1,角色扮演游戏,"角色扮演游戏, 冒险游戏, 策略游戏",informed,9,这个类型的游戏通常允许玩家做出影响故事走向和角色关系的抉择，带来更强的代入感。,1,NO,NaN
2709,1,Video Games,1,角色扮演游戏,"角色扮演游戏, 冒险游戏, 策略游戏",misinformed,1,一种数字媒体形式，通过引人入胜的叙事来讲述故事并娱乐观众，但类型定义较为宽泛。,1,NO,NaN


In [93]:
# Save translated data
lang_code = TARGET_LANGUAGE.value['code']
output_dir = Path(lang_code)
output_dir.mkdir(exist_ok=True)

# Generate output filename
if 'topic_category' in translated_df.columns and len(translated_df['topic_category'].unique()) == 1:
    topic = translated_df['topic_category'].iloc[0]
    output_file = output_dir / f"disinformer_clues({topic}).csv"
else:
    output_file = output_dir / f"disinformer_full_games_clues(topic).csv"

# Save to CSV
translated_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"✓ Translated data saved to: {output_file}")
print(f"  Total rows: {len(translated_df)}")
print(f"  File size: {output_file.stat().st_size / 1024:.1f} KB")

# Accumulate into master file
master_file = output_dir / f"disinformer_full_games_clues.csv"

# Check if master file exists and load it
if master_file.exists():
    master_df = pd.read_csv(master_file)
    print(f"\n✓ Loaded existing master file with {len(master_df)} rows")
    
    # Append new translated data
    master_df = pd.concat([master_df, translated_df], ignore_index=True)
    print(f"✓ Appended {len(translated_df)} new rows")
else:
    master_df = translated_df.copy()
    print(f"\n✓ Creating new master file")

# Remove duplicates based on all columns (keep first occurrence)
initial_count = len(master_df)
master_df = master_df.drop_duplicates(keep='first')
duplicates_removed = initial_count - len(master_df)
if duplicates_removed > 0:
    print(f"⚠️ Removed {duplicates_removed} duplicate rows")

# Save master file
master_df.to_csv(master_file, index=False, encoding='utf-8-sig')
print(f"\n✓ Master file saved to: {master_file}")
print(f"  Total rows: {len(master_df)}")
print(f"  File size: {master_file.stat().st_size / 1024:.1f} KB")

✓ Translated data saved to: ZH/disinformer_clues(Video Games).csv
  Total rows: 300
  File size: 59.3 KB

✓ Loaded existing master file with 2700 rows
✓ Appended 300 new rows

✓ Master file saved to: ZH/disinformer_full_games_clues.csv
  Total rows: 3000
  File size: 577.8 KB


In [94]:
# Quality check: Compare original and translated
def quality_check(original_df: pd.DataFrame, translated_df: pd.DataFrame, num_samples: int = 5):
    """
    Display side-by-side comparison of original and translated content.
    """
    print("Quality Check - Side-by-Side Comparison")
    print("="*120)
    
    for idx in range(min(num_samples, len(translated_df))):
        orig_row = original_df.iloc[idx]
        trans_row = translated_df.iloc[idx]
        
        print(f"\nSample {idx + 1}:")
        print("-"*120)
        print(f"Topic: {orig_row['topic_category']} | Round: {orig_row['round']} | Clue Type: {orig_row['clue_type']}")
        print(f"\nOriginal Answer: {orig_row['answer']}")
        print(f"Translated Answer: {trans_row['answer']}")
        print(f"\nOriginal Clue ({orig_row['word_count']} words):")
        print(f"  {orig_row['clue_text']}")
        print(f"\nTranslated Clue ({trans_row['word_count']} words):")
        print(f"  {trans_row['clue_text']}")
        print("-"*120)

# Run quality check if we have the original data
if 'df' in locals() and 'translated_df' in locals():
    quality_check(df, translated_df, num_samples=5)

Quality Check - Side-by-Side Comparison

Sample 1:
------------------------------------------------------------------------------------------------------------------------
Topic: Books | Round: 1 | Clue Type: informed

Original Answer: Fantasy
Translated Answer: 角色扮演游戏

Original Clue (16 words):
  This genre often features magic, mythical creatures, and imaginative worlds that defy the laws of reality.

Translated Clue (1 words):
  玩家通过获取经验、升级和获得新技能与物品来发展角色，是其核心机制。
------------------------------------------------------------------------------------------------------------------------

Sample 2:
------------------------------------------------------------------------------------------------------------------------
Topic: Books | Round: 1 | Clue Type: informed

Original Answer: Fantasy
Translated Answer: 角色扮演游戏

Original Clue (18 words):
  It typically involves quests, battles against evil, and the triumph of good over darkness in a fictional setting.

Translated Clue (1 words):
  玩家通常控制

## Usage Instructions

### To translate to a different language:

1. **Change the target language** in the "Language Configuration" cell:
   ```python
   TARGET_LANGUAGE = Language.FRENCH   # or RUSSIAN, CHINESE, ARABIC
   ```

2. **Restart kernel and run all cells** to translate with the new language

### To translate specific topics:

Modify the "Translate all data" cell:
```python
topic_to_translate = "Sports"  # Change to desired topic
df_topic = df[df['topic_category'] == topic_to_translate].copy()
translated_df = translate_dataframe(df_topic, delay_seconds=2.0)
```

### Available Topics:
- Books
- Broadcast_Media
- Food
- Inventions
- Nature
- Places
- Songs
- Sports
- Technology
- Video_Games

### Notes:
- Each topic has **10 games** × **2 rounds** × **15 clues** = **300 rows**
- Full dataset translation will make **~3000 API calls**
- Recommended: Translate one topic at a time
- Adjust `delay_seconds` based on API rate limits